Non SQL database

In [13]:
import csv
import requests

Saves all the data about fuel stations in one .csv file

In [15]:
url = 'https://community-nrel-national-renewable-energy-laboratory.p.rapidapi.com/api/alt-fuel-stations/v1/nearest.json'

location = '433 Bryant St., San Francisco'
fuel_type = 'ELEC'
limit = 50
offset = 0
data_list = []

headers = {
    'X-RapidAPI-Key': '51b07a06c6mshbaae7fd65806f5cp13da34jsn59c5aa35baa7',
    'X-RapidAPI-Host': 'community-nrel-national-renewable-energy-laboratory.p.rapidapi.com'
}

while True:
    querystring = {'location': location, 'fuel_type': fuel_type, 'limit': limit, 'offset': offset}
    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()['fuel_stations']
        if not data:
            break
        data_list.extend(data)
        offset += limit
    else:
        print(f'Failed to get data for offset {offset}. Skipping...')
        break

csv_file_path = 'fuel_stations_data.csv'

if data_list:
    keys = data_list[0].keys()

    keys = list(keys) + ['federal_agency']

    with open(csv_file_path, mode='w', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data_list)

    print(f'Data has been saved to {csv_file_path}.')
else:
    print('No data received from the API.')


Data has been saved to fuel_stations_data.csv.


Sorting the data about fuel stations into three groups (Public, Private, Others) and saving the data in separate .csv files.

In [23]:
import csv
import requests

url = "https://community-nrel-national-renewable-energy-laboratory.p.rapidapi.com/api/alt-fuel-stations/v1/nearest.json"

location = "433 Bryant St., San Francisco"
fuel_type = "ELEC"
limit = 50
offset = 0
public_data = []
private_data = []
others_data = []

headers = {
    "X-RapidAPI-Key": "51b07a06c6mshbaae7fd65806f5cp13da34jsn59c5aa35baa7",
    "X-RapidAPI-Host": "community-nrel-national-renewable-energy-laboratory.p.rapidapi.com"
}

while True:
    querystring = {"location": location, "fuel_type": fuel_type, "limit": limit, "offset": offset}
    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()["fuel_stations"]
        if not data:
            break

        for station in data:
            group = station.get('groups_with_access_code', '').lower()
            if group == 'public':
                public_data.append(station)
            elif group == 'private':
                private_data.append(station)
            elif group != 'private' or group != 'public':
                others_data.append(station)

        offset += limit
    else:
        print(f"Failed to get data for offset {offset}. Skipping...")
        break

public_csv_file = "public_stations_data.csv"
private_csv_file = "private_stations_data.csv"
others_csv_file = "others_stations_data.csv"

if public_data:
    keys = list(public_data[0].keys()) + ['federal_agency', 'ev_network_ids']

    # Writing data to the 'Public' CSV file
    with open(public_csv_file, mode="w", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(public_data)

    print(f"Public stations data has been saved to {public_csv_file}.")

if private_data:
    keys = list(private_data[0].keys()) + ['federal_agency', 'ev_network_ids']

    # Writing data to the 'Private' CSV file
    with open(private_csv_file, mode="w", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(private_data)

    print(f"Private stations data has been saved to {private_csv_file}.")

if others_data:
    keys = list(others_data[0].keys()) + ['federal_agency', 'ev_network_ids']

    # Writing data to the 'Others' CSV file
    with open(others_csv_file, mode="w", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(others_data)

    print(f"Other stations data has been saved to {others_csv_file}.")


Public stations data has been saved to public_stations_data.csv.
Private stations data has been saved to private_stations_data.csv.
Other stations data has been saved to others_stations_data.csv.
